In [2]:
import numpy as np
import pandas as pd

import plotly.graph_objs as go

from ostk.physics.units import Length
from ostk.physics.units import Angle
from ostk.physics.time import Scale
from ostk.physics.time import Instant
from ostk.physics.time import Duration
from ostk.physics.time import Interval
from ostk.physics.time import DateTime
from ostk.physics.coordinate.spherical import LLA
from ostk.physics.coordinate import Frame
from ostk.physics import Environment
from ostk.physics.environment.objects.celestial_bodies import Earth

from ostk.astrodynamics import Trajectory
from ostk.astrodynamics.trajectory import Orbit
from ostk.astrodynamics.trajectory.orbit.models import Kepler
from ostk.astrodynamics.trajectory.orbit.models.kepler import COE
from ostk.astrodynamics.trajectory.orbit.models import SGP4
from ostk.astrodynamics.trajectory.orbit.models.sgp4 import TLE

from utils import propagator

/srv/conda/envs/notebook/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

to-Python converter for ostk::physics::time::DateTime already registered; second conversion method ignored.

/srv/conda/envs/notebook/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

to-Python converter for ostk::math::obj::IntervalBase::Type already registered; second conversion method ignored.



In [3]:
environment = Environment.default()

In [13]:
epoch = Instant.date_time(DateTime(2020, 1, 1, 0, 0, 0), Scale.UTC)
earth = environment.access_celestial_object_with_name("Earth")

start_instant = Instant.date_time(DateTime(2020, 1, 1, 0, 0, 0), Scale.UTC)
end_instant   = Instant.date_time(DateTime(2020, 1, 1, 1, 0, 0), Scale.UTC)
interval = Interval.closed(start_instant, end_instant)
step = Duration.minutes(1.0)
instants = interval.generate_grid(step)

In [14]:
def convert_state (instant, state):
    
    lla = LLA.cartesian(state.get_position().in_frame(Frame.ITRF(), state.get_instant()).get_coordinates(), Earth.equatorial_radius, Earth.flattening)
    
    return [
                repr(instant),
                float(instant.get_modified_julian_date(Scale.UTC)),
                *state.get_position().get_coordinates().transpose()[0].tolist(),
                *state.get_velocity().get_coordinates().transpose()[0].tolist(),
                float(lla.get_latitude().in_degrees()),
                float(lla.get_longitude().in_degrees()),
                float(lla.get_altitude().in_meters())
            ]

In [37]:
G = {}

a0 = Length.kilometers(7000.0)
e0 = 0.0001
i0 = Angle.degrees(35.0)
raan0 = Angle.degrees(40.0)
aop0 = Angle.degrees(45.0)
nu0 = Angle.degrees(50.0)
G['0'] = [a0, e0, i0, raan0, aop0, nu0]

#Satellite GPS1
a1 = Length.kilometers(26559.8)
e1 = 10**-7
i1 = Angle.degrees(55.0)
raan1 = Angle.degrees(48.85)
aop1 = Angle.degrees(0.0)
nu1 = Angle.degrees(83.29)
G['1'] = [a1, e1, i1, raan1, aop1, nu1]

#Satellite GPS2
a2 = Length.kilometers(26559.8)
e2 = 10**-7
i2 = Angle.degrees(55.0)
raan2 = Angle.degrees(48.85)
aop2 = Angle.degrees(0.0)
nu2 = Angle.degrees(343.21)
G['2'] = [a2, e2, i2, raan2, aop2, nu2]

#Satellite GPS3
a3 = Length.kilometers(26559.8)
e3 = 10**-7
i3 = Angle.degrees(55.0)
raan3 = Angle.degrees(48.85)
aop3 = Angle.degrees(0.0)
nu3 = Angle.degrees(311.08)
G['3'] = [a3, e3, i3, raan3, aop3, nu3]

#Satellite GPS3
a4 = Length.kilometers(26559.8)
e4 = 10**-7
i4 = Angle.degrees(55.0)
raan4 = Angle.degrees(48.85)
aop4 = Angle.degrees(0.0)
nu4 = Angle.degrees(212.97)
G['4'] = [a4, e4, i4, raan4, aop4, nu4]

n = len(G)


def orbit_gen(G, k):
    a, e, i, raan, aop, nu = G[str(k)]
    coe = COE(a, e, i, raan, aop, nu)
    model = Kepler(coe, epoch, earth, Kepler.PerturbationType.No)
    orbit = Orbit(model, earth)
    states = [[instant, orbit.get_state_at(instant)] for instant in instants]
    orbit_data = [convert_state(instant, state) for [instant, state] in states]
    orbit_data = np.array(orbit_data)
    orbit_df = pd.DataFrame(orbit_data, columns= \
    ['$Time^{UTC}$', '$MJD^{UTC}$', '$x_{x}^{ECI}$', '$x_{y}^{ECI}$', '$x_{z}^{ECI}$', '$v_{x}^{ECI}$', '$v_{y}^{ECI}$', '$v_{z}^{ECI}$', '$Latitude$', '$Longitude$', '$Altitude$'])
    return orbit_df


In [38]:
coe1 = COE(a1, e1, i1, raan1, aop1, nu1)
model1 = Kepler(coe1, epoch, earth, Kepler.PerturbationType.No)
orbit1 = Orbit(model1, earth)
model1.get_perturbation_type()
model1.get_classical_orbital_elements()

-- Classical Orbital Elements ----------------------------------------------------------------------
    Semi-major axis:                         26559800.0 [m]                           
    Eccentricity:                            9.9999999999999995e-08                   
    Inclination:                             55.0 [deg]                               
    Right ascension of the ascending node:   48.850000000000001 [deg]                 
    Argument of periapsis:                   0.0 [deg]                                
    True anomaly:                            83.290000000000006 [deg]                 
----------------------------------------------------------------------------------------------------

In [39]:
orbit_df = {}

for i in range(n):
    orbit_df[str(i)] = orbit_gen(G, i)

In [40]:
figure = go.Figure()

for i in range(n):
    
    orbit_sat = orbit_df[str(i)]
    color = 'blue'
    
    if i == 0:
        color = 'red'
    
    figure.add_trace(go.Scattergeo(
            lon = orbit_sat['$Longitude$'],
            lat = orbit_sat['$Latitude$'],
            mode = 'lines',
            line = go.scattergeo.Line(
                width = 1,
                color = color
            )
        ))

    
    
figure.update_layout(
        title = None,
        showlegend = False,
        height=1000,
        geo = go.layout.Geo(
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)'
        )
    )

figure.show()